In [25]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
# 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 기본값으로 설정 (처음 5개와 마지막 5개 행만 출력)
pd.reset_option('display.max_rows')

# 출력 포맷 설정 (소수점 4자리까지)
pd.options.display.float_format = '{:.4f}'.format

import platform
import seaborn as sns

import matplotlib.pyplot as plt

# 운영 체제 확인
if platform.system() == 'Darwin':  # Mac
    print('apple gothic')
    font_name = 'AppleGothic'
elif platform.system() == 'Windows':  # Windows
    font_name = 'NanumGothic'
else:
    font_name = None

# 한글 폰트 설정
if font_name:
    plt.rcParams['font.family'] = font_name

# 마이너스 부호 설정
plt.rcParams['axes.unicode_minus'] = False

apple gothic


## 데이터 로드

In [26]:
df = pd.read_csv(
    '../data/2_재무데이터결합.csv', 
    dtype = {'거래소코드' : 'object'}
)

df.shape

(22245, 86)

## 이자보상배율 계산

### 결측치 대체 (연결 데이터를 개별 데이터로 대체)

In [27]:
# 연결 데이터가 결측치인 경우, 개별 데이터로 대체

df['영업손익'] = df['[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS연결)(천원)'].fillna(
    df['[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS)(천원)']
)
df['이자비용1'] = df['[U01B470010000][제조]   이자비용(IFRS연결)(천원)'].fillna(
    df['[U01B470010000][제조]   이자비용(IFRS)(천원)']
)
df['이자비용2'] = df['[U01B550010000][제조]   이자비용(IFRS연결)(천원)'].fillna(
    df['[U01B550010000][제조]   이자비용(IFRS)(천원)']
)
df['이자비용3'] = df['[U01B201013300][제조]   이자비용(IFRS연결)(천원)'].fillna(
    df['[U01B201013300][제조]   이자비용(IFRS)(천원)']
)
df['이자비용4'] = df['[U01B350016400][제조]   이자비용(IFRS연결)(천원)'].fillna(
    df['[U01B350016400][제조]   이자비용(IFRS)(천원)']
)

df['이자보상배율'] = df['[제조]이자보상배율(이자비용)(IFRS연결)'].fillna(
    df['[제조]이자보상배율(이자비용)(IFRS)']
)

df[['회사명', '거래소코드', '회계년도', '결산년도', 'market', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4']].isna().sum()

회사명          0
거래소코드        0
회계년도         0
결산년도         0
market       0
영업손익       235
이자비용1     3249
이자비용2      235
이자비용3      235
이자비용4      235
dtype: int64

### 이자비용 확인

In [28]:
# 이자비용1, 이자비용2 중 음수(-)값 존재
# 이자비용2를 제외하고 대부분 0 값

df[['영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4']].describe()

,영업손익,이자비용1,이자비용2,이자비용3,이자비용4
count,22010.0000,18996.0000,22010.0000,22010.0000,22010.0000
mean,76099541.9120,33561.6712,13162300.7032,26379.4278,1075.8290
std,1005592981.8084,1310686.4972,77052167.1081,915823.6352,110343.1917
min,-32655153000.0000,-46603.0000,-27580.0000,0.0000,0.0000
25%,-125461.7500,0.0000,253280.7500,0.0000,0.0000
50%,5397912.5000,0.0000,1180028.0000,0.0000,0.0000
75%,21852836.0000,0.0000,4038425.5000,0.0000,0.0000
max,58886669000.0000,108177355.0000,2818546000.0000,55966389.0000,16157000.0000


#### 이자비용2 : 금융원가(U01B550010000)

In [29]:
# 이자보상배율2 = 영업손익 / 이자비용2

df['이자보상배율2'] = (df['영업손익'] / df['이자비용2']).round(2)

In [30]:
df[['영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4']].isna().sum()

영업손익      235
이자비용1    3249
이자비용2     235
이자비용3     235
이자비용4     235
dtype: int64

#### 이자비용1 : 기타손실(U01B470010000)

In [31]:
# 이자비용1이 0이 아닌 경우
# 이자보상배율 != 이자보상배율2

df.loc[
    (df['이자비용1'].notna()) & (df['이자비용1']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율2']
].head()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율2
190,(주)대유플러스,000300,2016/12,10364596.0000,103233.0000,3877760.0000,0.0000,0.0000,2.6000,2.6700
191,(주)대유플러스,000300,2017/12,6882656.0000,210656.0000,3274868.0000,0.0000,0.0000,1.9700,2.1000
192,(주)대유플러스,000300,2018/12,36513814.0000,199786.0000,5753961.0000,0.0000,0.0000,6.1300,6.3500
193,(주)대유플러스,000300,2019/12,16306666.0000,755262.0000,5518406.0000,0.0000,0.0000,2.6000,2.9500
194,(주)대유플러스,000300,2020/12,14542187.0000,375644.0000,6512533.0000,0.0000,0.0000,2.1100,2.2300


In [32]:
# 이자비용1이 0이 아닐 경우,
# 이자보상배율 = 영업손익 / (이자비용1+이자비용2)

df['이자비용1+이자비용2'] = df['이자비용1']+df['이자비용2']
df['이자보상배율12'] = (df['영업손익'] / df['이자비용1+이자비용2']).round(2)

df.loc[
    (df['이자비용1'].notna()) & (df['이자비용1']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율12']
].head()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율12
190,(주)대유플러스,000300,2016/12,10364596.0000,103233.0000,3877760.0000,0.0000,0.0000,2.6000,2.6000
191,(주)대유플러스,000300,2017/12,6882656.0000,210656.0000,3274868.0000,0.0000,0.0000,1.9700,1.9700
192,(주)대유플러스,000300,2018/12,36513814.0000,199786.0000,5753961.0000,0.0000,0.0000,6.1300,6.1300
193,(주)대유플러스,000300,2019/12,16306666.0000,755262.0000,5518406.0000,0.0000,0.0000,2.6000,2.6000
194,(주)대유플러스,000300,2020/12,14542187.0000,375644.0000,6512533.0000,0.0000,0.0000,2.1100,2.1100


#### 이자비용3 : 특수관계자매출원가(U01B201013300)

In [33]:
# 이자비용3이 0이 아닌 경우,
# 이자보상배율 = 영업손익 / 이자비용2

df.loc[
    (df['이자비용3'].notna()) & (df['이자비용3']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율2']
].head()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율2
2520,(주)한창,005110,2011/12,-787616.0000,NaN,284571.0000,51616.0000,0.0000,-2.7700,-2.7700
2521,(주)한창,005110,2012/12,1137778.0000,NaN,138868.0000,50000.0000,0.0000,8.1900,8.1900
3934,무림페이퍼(주),009200,2018/12,124085429.0000,0.0000,30209775.0000,7163971.0000,0.0000,4.1100,4.1100
3935,무림페이퍼(주),009200,2019/12,68821695.0000,0.0000,32153808.0000,7367344.0000,0.0000,2.1400,2.1400
3936,무림페이퍼(주),009200,2020/12,27266032.0000,0.0000,30821929.0000,8880129.0000,0.0000,0.8800,0.8800


In [34]:
# 이자비용3이 0이 아닐 경우,
# 이자보상배율 = 영업손익 / (이자비용2)
# 이자보상배율 != 영업손익 / (이자비용3+이자비용2)

df['이자비용3+이자비용2'] = df['이자비용3']+df['이자비용2']
df['이자보상배율32'] = (df['영업손익'] / df['이자비용3+이자비용2']).round(2)

df.loc[
    (df['이자비용3'].notna()) & (df['이자비용3']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율2', '이자보상배율32']
].head()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율2,이자보상배율32
2520,(주)한창,005110,2011/12,-787616.0000,NaN,284571.0000,51616.0000,0.0000,-2.7700,-2.7700,-2.3400
2521,(주)한창,005110,2012/12,1137778.0000,NaN,138868.0000,50000.0000,0.0000,8.1900,8.1900,6.0200
3934,무림페이퍼(주),009200,2018/12,124085429.0000,0.0000,30209775.0000,7163971.0000,0.0000,4.1100,4.1100,3.3200
3935,무림페이퍼(주),009200,2019/12,68821695.0000,0.0000,32153808.0000,7367344.0000,0.0000,2.1400,2.1400,1.7400
3936,무림페이퍼(주),009200,2020/12,27266032.0000,0.0000,30821929.0000,8880129.0000,0.0000,0.8800,0.8800,0.6900


#### 이자비용4 : 기타(영업)비용(U01B350016400)

In [35]:
# 이자비용4가 0이 아닌 경우,
# 이자보상배율 != 이자보상배율2
df.loc[
    (df['이자비용4'].notna()) & (df['이자비용4']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율2']
].tail()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율2
14862,(주)티비에이치글로벌,084870,2011/12,33301972.0000,NaN,7625537.0000,0.0000,5071.0000,4.3600,4.3700
15246,(주)케이씨티,089150,2011/12,1910441.0000,NaN,0.0000,0.0000,4181.0000,456.9300,inf
16033,(주)네오위즈,095660,2011/12,106795298.0000,NaN,6316638.0000,0.0000,19192.0000,16.8600,16.9100
17765,(주)와이지엔터테인먼트,122870,2021/12,50561444.0000,0.0000,795111.0000,0.0000,14230.0000,62.4700,63.5900
17766,(주)와이지엔터테인먼트,122870,2022/12,46639449.0000,0.0000,433068.0000,0.0000,16914.0000,103.6500,107.7000


In [36]:
# 이자비용4이 0이 아닐 경우,
# 이자보상배율 = 영업손익 / (이자비용4+이자비용2)

df['이자비용4+이자비용2'] = df['이자비용4']+df['이자비용2']
df['이자보상배율42'] = (df['영업손익'] / df['이자비용4+이자비용2']).round(2)

df.loc[
    (df['이자비용4'].notna()) & (df['이자비용4']!=0),
    ['회사명', '거래소코드', '회계년도', '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', '이자보상배율42']
].tail()

,회사명,거래소코드,회계년도,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,이자보상배율,이자보상배율42
14862,(주)티비에이치글로벌,084870,2011/12,33301972.0000,NaN,7625537.0000,0.0000,5071.0000,4.3600,4.3600
15246,(주)케이씨티,089150,2011/12,1910441.0000,NaN,0.0000,0.0000,4181.0000,456.9300,456.9300
16033,(주)네오위즈,095660,2011/12,106795298.0000,NaN,6316638.0000,0.0000,19192.0000,16.8600,16.8600
17765,(주)와이지엔터테인먼트,122870,2021/12,50561444.0000,0.0000,795111.0000,0.0000,14230.0000,62.4700,62.4700
17766,(주)와이지엔터테인먼트,122870,2022/12,46639449.0000,0.0000,433068.0000,0.0000,16914.0000,103.6500,103.6500


### `총이자비용` = 이자비용1 + 이자비용2 + 이자비용4

In [37]:
df['총이자비용'] = df[['이자비용1', '이자비용2', '이자비용4']].sum(axis=1, min_count=1)

In [38]:
# 이자비용이 음수(-)인 경우

df.loc[
    (df[['이자비용1', '이자비용2', '이자비용3', '이자비용4']]<0).any(axis=1),
    ['회사명', '거래소코드', '회계년도', '결산년도', 'market', 
     '영업손익', '이자비용1', '이자비용2', '이자비용3', '이자비용4', '총이자비용', '이자보상배율']
]

,회사명,거래소코드,회계년도,결산년도,market,영업손익,이자비용1,이자비용2,이자비용3,이자비용4,총이자비용,이자보상배율
2184,(주)삼일씨엔에스,004440,2018/12,2018,KOSPI,3734245.0000,-46603.0000,288744.0000,0.0000,0.0000,242141.0000,15.4200
2185,(주)삼일씨엔에스,004440,2019/12,2019,KOSPI,3576913.0000,-25942.0000,76202.0000,0.0000,0.0000,50260.0000,71.1700
12636,(주)에스아이리소스,065420,2021/12,2021,KOSDAQ,2002252.0000,0.0000,-27580.0000,0.0000,0.0000,-27580.0000,999999999.0000


In [39]:
# (이자비용<0 이지만 총이자비용>0인 경우)

# (주)삼일씨엔에스, 2018/12
# 이자보상배율 15.4200
# 영업손익 3734245.0000
# 총이자비용 242141.0000 (>0)

print(f"이자보상배율 : 15.42")

print('영업손익/총이자비용 :', np.round(3734245.0000/242141.0000, 2))

이자보상배율 : 15.42
영업손익/총이자비용 : 15.42


In [40]:
# (총이자비용<0인 경우)

# (주)에스아이리소스, 065420
# 이자보상배율 999999999.0000
# 영업손익 2002252.0000
# 총이자비용 -242141.0000 (<0)

print(f"이자보상배율 : 999999999.0000")

print('영업손익/총이자비용 :', np.round(2002252.0000/-242141.0000, 2))

이자보상배율 : 999999999.0000
영업손익/총이자비용 : -8.27


In [41]:
# 총이자비용 < 0 은 0으로 대체
df.loc[df['총이자비용']<0, '총이자비용'] = 0

### `이자보상배율` = 영업손익 / 총이자비용

In [42]:
df['이자보상배율_new'] = (df['영업손익'] / df['총이자비용']).round(2)

In [43]:
df['이자보상배율_new'].isna().sum()

235

In [44]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '[제조]이자보상배율(이자비용)(IFRS연결)',
       '[제조]이자보상배율(이자비용)(IFRS)',
       '[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS연결)(천원)',
       '[U01B470010000][제조]   이자비용(IFRS연결)(천원)',
       '[U01B550010000][제조]   이자비용(IFRS연결)(천원)',
       ...
       '이자보상배율', '이자보상배율2', '이자비용1+이자비용2', '이자보상배율12', '이자비용3+이자비용2',
       '이자보상배율32', '이자비용4+이자비용2', '이자보상배율42', '총이자비용', '이자보상배율_new'],
      dtype='object', length=101)

In [45]:
df.shape

(22245, 101)

In [46]:
# 불필요한 컬럼 제거
drop_cols = [
    '[제조]이자보상배율(이자비용)(IFRS연결)',
    '[제조]이자보상배율(이자비용)(IFRS)',
    '[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS연결)(천원)',
    '[U01B470010000][제조]   이자비용(IFRS연결)(천원)',
    '[U01B550010000][제조]   이자비용(IFRS연결)(천원)',
    '[U01B201013300][제조]   이자비용(IFRS연결)(천원)',
    '[U01B350016400][제조]   이자비용(IFRS연결)(천원)',
    '[U01B420000000][제조]* (정상)영업손익(보고서기재)(IFRS)(천원)',
    '[U01B470010000][제조]   이자비용(IFRS)(천원)',
    '[U01B550010000][제조]   이자비용(IFRS)(천원)',
    '[U01B201013300][제조]   이자비용(IFRS)(천원)',
    '[U01B350016400][제조]   이자비용(IFRS)(천원)',
    '이자비용1', '이자비용2', '이자비용3', '이자비용4', '이자보상배율', 
    
    '이자비용1+이자비용2', '이자보상배율12', '이자비용3+이자비용2', '이자보상배율32',
    '이자보상배율2', '이자비용4+이자비용2', '이자보상배율42', '총이자비용'
]

df = df.drop(columns = drop_cols)

df.shape

(22245, 76)

In [47]:
# 이자보상배율_new 컬럼명 변경

df.rename(columns = {'이자보상배율_new' : '이자보상배율'}, inplace=True)

## 파일로 저장

In [48]:
df.to_csv('../data/3_이자보상배율계산.csv', index=None)